# Fight Prediction

In [159]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import sys, warnings, os
from sklearn.dummy import DummyClassifier

In [160]:
pd.set_option("display.max_columns", None)

In [161]:
df = pd.read_csv("data/ufc-master.csv")

df

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,title_bout,weight_class,gender,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_SIG_STR_landed,B_avg_SIG_STR_pct,B_avg_SUB_ATT,B_avg_TD_landed,B_avg_TD_pct,B_longest_win_streak,B_losses,B_total_rounds_fought,B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins,B_Stance,B_Height_cms,B_Reach_cms,B_Weight_lbs,R_current_lose_streak,R_current_win_streak,R_draw,R_avg_SIG_STR_landed,R_avg_SIG_STR_pct,R_avg_SUB_ATT,R_avg_TD_landed,R_avg_TD_pct,R_longest_win_streak,R_losses,R_total_rounds_fought,R_total_title_bouts,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,R_age,B_age,lose_streak_dif,win_streak_dif,longest_win_streak_dif,win_dif,loss_dif,total_round_dif,total_title_bout_dif,ko_dif,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,constant_1,B_match_weightclass_rank,R_match_weightclass_rank,R_Women's Flyweight_rank,R_Women's Featherweight_rank,R_Women's Strawweight_rank,R_Women's Bantamweight_rank,R_Heavyweight_rank,R_Light Heavyweight_rank,R_Middleweight_rank,R_Welterweight_rank,R_Lightweight_rank,R_Featherweight_rank,R_Bantamweight_rank,R_Flyweight_rank,R_Pound-for-Pound_rank,B_Women's Flyweight_rank,B_Women's Featherweight_rank,B_Women's Strawweight_rank,B_Women's Bantamweight_rank,B_Heavyweight_rank,B_Light Heavyweight_rank,B_Middleweight_rank,B_Welterweight_rank,B_Lightweight_rank,B_Featherweight_rank,B_Bantamweight_rank,B_Flyweight_rank,B_Pound-for-Pound_rank,better_rank,finish,finish_details,finish_round,finish_round_time,total_fight_time_secs,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds
0,Thiago Santos,Johnny Walker,-150.0,130,66.666667,130.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Red,False,Light Heavyweight,MALE,5,0,1,0,3.420000,0.59,0.700000,0.240000,1.00,4,2,11,0,0,0,1,4,0,0,5,Orthodox,198.12,208.28,205,3,0,0,3.950000,0.480000,0.100000,0.610000,0.360,4,8,43,1,0,0,2,11,0,0,13,Orthodox,187.96,193.04,205,37,29,-3,1,0,-8,-6,-32,-1,-7,0,10.16,15.24,-8,-0.530000,0.600000,-0.370000,1,1,10.0,5.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Red,U-DEC,NaN,5.0,5:00,1500.0,800.0,900.0,2000.0,1600.0,-110.0,175.0
1,Alex Oliveira,Niko Price,170.0,-200,170.000000,50.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Blue,False,Welterweight,MALE,3,2,0,0,5.160000,0.42,0.800000,0.790000,0.22,2,5,24,0,0,0,0,4,2,0,6,Orthodox,182.88,193.04,170,2,0,0,2.970000,0.510000,0.500000,2.270000,0.370,4,8,44,0,0,1,3,4,3,0,11,Orthodox,180.34,193.04,170,33,32,0,0,-2,-5,-3,-20,0,0,-1,2.54,0.00,-1,2.190000,0.300000,-1.480000,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,U-DEC,NaN,3.0,5:00,900.0,450.0,350.0,700.0,1100.0,550.0,120.0
2,Misha Cirkunov,Krzysztof Jotko,110.0,-130,110.000000,76.923077,2021-10-02,"Las Vegas, Nevada, USA",USA,Blue,False,Middleweight,MALE,3,1,0,0,2.920000,0.41,0.100000,1.150000,0.34,5,5,38,0,0,2,6,1,0,0,9,Southpaw,185.42,195.58,185,1,0,0,3.770000,0.490000,1.700000,4.480000,0.440,4,4,13,0,0,0,0,1,5,0,6,Orthodox,190.50,195.58,205,34,32,0,0,1,3,1,25,0,0,-5,-5.08,0.00,-2,-0.850000,-1.600000,-3.330000,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,S-DEC,NaN,3.0,5:00,900.0,550.0,275.0,275.0,1400.0,600.0,185.0
3,Alexander Hernandez,Mike Breeden,-675.0,475,14.814815,475.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Red,False,Lightweight,MALE,3,1,0,0,4.040000,0.34,0.000000,0.000000,0.00,0,1,3,0,0,0,0,0,0,0,0,Orthodox,177.80,177.80,155,1,0,0,3.790000,0.360000,0.000000,1.570000,0.350,2,3,15,0,0,0,2,2,0,0,4,Orthodox,17

In [162]:
#Aggregate the rank into a single columns 

def get_rank_r(row):
    ranks = row["R_Women's Flyweight_rank"], row["R_Women's Featherweight_rank"], row["R_Women's Strawweight_rank"], row["R_Women's Bantamweight_rank"], row["R_Heavyweight_rank"], row["R_Light Heavyweight_rank"], row["R_Middleweight_rank"], row["R_Welterweight_rank"], row["R_Lightweight_rank"], row["R_Featherweight_rank"], row["R_Bantamweight_rank"], row["R_Flyweight_rank"], row["R_Pound-for-Pound_rank"], row["R_match_weightclass_rank"]
    ranks = list(ranks)
    nan_bool = np.isnan(ranks)
    weights = [1 if ~n else 0 for n in nan_bool]
    masked_data = np.ma.masked_array(ranks, np.isnan(ranks))
    if sum(weights):
        return np.average(masked_data, weights=weights, axis=0)
    if not sum(weights):
        return np.nan
def get_rank_b(row):
    ranks = row["B_Women's Flyweight_rank"], row["B_Women's Featherweight_rank"], row["B_Women's Strawweight_rank"], row["B_Women's Bantamweight_rank"], row["B_Heavyweight_rank"], row["B_Light Heavyweight_rank"], row["B_Middleweight_rank"], row["B_Welterweight_rank"], row["B_Lightweight_rank"], row["B_Featherweight_rank"], row["B_Bantamweight_rank"], row["B_Flyweight_rank"], row["B_Pound-for-Pound_rank"], row["B_match_weightclass_rank"]
    ranks = list(ranks)
    nan_bool = np.isnan(ranks)
    weights = [1 if ~n else 0 for n in nan_bool]
    masked_data = np.ma.masked_array(ranks, np.isnan(ranks))
    if sum(weights):
        return np.average(masked_data, weights=weights, axis=0)
    if not sum(weights):
        return np.nan
    

df["R_rank"] = df.apply(lambda row: get_rank_r(row), axis=1)
df["B_rank"] = df.apply(lambda row: get_rank_b(row), axis=1)


In [163]:
# Drop all those now useless columns
print(df.shape)
df = df.drop(["R_Women's Flyweight_rank","R_Women's Featherweight_rank","R_Women's Strawweight_rank","R_Women's Bantamweight_rank","R_Heavyweight_rank","R_Light Heavyweight_rank","R_Middleweight_rank","R_Welterweight_rank","R_Lightweight_rank","R_Featherweight_rank","R_Bantamweight_rank","R_Flyweight_rank", "R_Pound-for-Pound_rank", "R_match_weightclass_rank"], axis=1)
df = df.drop(["B_Women's Flyweight_rank","B_Women's Featherweight_rank","B_Women's Strawweight_rank","B_Women's Bantamweight_rank","B_Heavyweight_rank","B_Light Heavyweight_rank","B_Middleweight_rank","B_Welterweight_rank","B_Lightweight_rank","B_Featherweight_rank","B_Bantamweight_rank","B_Flyweight_rank", "B_Pound-for-Pound_rank", "B_match_weightclass_rank"], axis=1)

print(df.shape)
df

(4896, 121)
(4896, 93)


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,title_bout,weight_class,gender,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_SIG_STR_landed,B_avg_SIG_STR_pct,B_avg_SUB_ATT,B_avg_TD_landed,B_avg_TD_pct,B_longest_win_streak,B_losses,B_total_rounds_fought,B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins,B_Stance,B_Height_cms,B_Reach_cms,B_Weight_lbs,R_current_lose_streak,R_current_win_streak,R_draw,R_avg_SIG_STR_landed,R_avg_SIG_STR_pct,R_avg_SUB_ATT,R_avg_TD_landed,R_avg_TD_pct,R_longest_win_streak,R_losses,R_total_rounds_fought,R_total_title_bouts,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,R_age,B_age,lose_streak_dif,win_streak_dif,longest_win_streak_dif,win_dif,loss_dif,total_round_dif,total_title_bout_dif,ko_dif,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,constant_1,better_rank,finish,finish_details,finish_round,finish_round_time,total_fight_time_secs,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds,R_rank,B_rank
0,Thiago Santos,Johnny Walker,-150.0,130,66.666667,130.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Red,False,Light Heavyweight,MALE,5,0,1,0,3.420000,0.59,0.700000,0.240000,1.00,4,2,11,0,0,0,1,4,0,0,5,Orthodox,198.12,208.28,205,3,0,0,3.950000,0.480000,0.100000,0.610000,0.360,4,8,43,1,0,0,2,11,0,0,13,Orthodox,187.96,193.04,205,37,29,-3,1,0,-8,-6,-32,-1,-7,0,10.16,15.24,-8,-0.530000,0.600000,-0.370000,1,1,Red,U-DEC,NaN,5.0,5:00,1500.0,800.0,900.0,2000.0,1600.0,-110.0,175.0,5.0,10.0
1,Alex Oliveira,Niko Price,170.0,-200,170.000000,50.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Blue,False,Welterweight,MALE,3,2,0,0,5.160000,0.42,0.800000,0.790000,0.22,2,5,24,0,0,0,0,4,2,0,6,Orthodox,182.88,193.04,170,2,0,0,2.970000,0.510000,0.500000,2.270000,0.370,4,8,44,0,0,1,3,4,3,0,11,Orthodox,180.34,193.04,170,33,32,0,0,-2,-5,-3,-20,0,0,-1,2.54,0.00,-1,2.190000,0.300000,-1.480000,1,1,neither,U-DEC,NaN,3.0,5:00,900.0,450.0,350.0,700.0,1100.0,550.0,120.0,NaN,NaN
2,Misha Cirkunov,Krzysztof Jotko,110.0,-130,110.000000,76.923077,2021-10-02,"Las Vegas, Nevada, USA",USA,Blue,False,Middleweight,MALE,3,1,0,0,2.920000,0.41,0.100000,1.150000,0.34,5,5,38,0,0,2,6,1,0,0,9,Southpaw,185.42,195.58,185,1,0,0,3.770000,0.490000,1.700000,4.480000,0.440,4,4,13,0,0,0,0,1,5,0,6,Orthodox,190.50,195.58,205,34,32,0,0,1,3,1,25,0,0,-5,-5.08,0.00,-2,-0.850000,-1.600000,-3.330000,1,1,neither,S-DEC,NaN,3.0,5:00,900.0,550.0,275.0,275.0,1400.0,600.0,185.0,13.0,NaN
3,Alexander Hernandez,Mike Breeden,-675.0,475,14.814815,475.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Red,False,Lightweight,MALE,3,1,0,0,4.040000,0.34,0.000000,0.000000,0.00,0,1,3,0,0,0,0,0,0,0,0,Orthodox,177.80,177.80,155,1,0,0,3.790000,0.360000,0.000000,1.570000,0.350,2,3,15,0,0,0,2,2,0,0,4,Orthodox,175.26,182.88,155,29,32,0,0,-2,-4,-2,-12,0,-2,0,2.54,-5.08,3,0.250000,0.000000,-1.570000,1,1,neither,KO/TKO,Punch,1.0,1:20,80.0,175.0,900.0,500.0,3500.0,110.0,1100.0,NaN,NaN
4,Joe Solecki,Jared Gordon,-135.0,115,74.074074,115.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Blue,False,Lightweight,MALE,3,0,2,0,5.220000,0.56,0.000000,2.540000,0.39,2,3,19,0,0,0,4,1,0,0,5,Orthodox,175.26,172.72,145,0,4,0,2.640000,0.620000,0.600000,2.850000,0.520,4,0,8,0,0,0,2,0,2,0,4,Orthodox,175.26,177.80,155,28,33,0,-2,-2,1,3,11,0,1,-2,0.00,-5.08,5,2.580000,-0.600000,-0.310000,1,1,neither,S-DEC,NaN,3.0,5:00,900.0,165.0,200.0,400.0,1200.0,900.0,600.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [164]:
categorical_features = [col for col in df.columns if df[col].dtypes == 'object']
numeric_features = [col for col in df.columns if col not in categorical_features]
print(f"Categorical feature: {categorical_features}")
print(f"Numerical feature: {numeric_features}")
#TODO check features (title_bout)


Categorical feature: ['R_fighter', 'B_fighter', 'date', 'location', 'country', 'Winner', 'weight_class', 'gender', 'B_Stance', 'R_Stance', 'better_rank', 'finish', 'finish_details', 'finish_round_time']
Numerical feature: ['R_odds', 'B_odds', 'R_ev', 'B_ev', 'title_bout', 'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak', 'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT', 'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses', 'B_total_rounds_fought', 'B_total_title_bouts', 'B_win_by_Decision_Majority', 'B_win_by_Decision_Split', 'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission', 'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Height_cms', 'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak', 'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed', 'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct', 'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought', 'R_total_title_bouts', 'R_win_by_De

In [165]:
## Get label
label = df.Winner
label = [1 if win == 'Red' else 0 if win == 'Blue' else np.nan for win in label]
print(label)

[1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 

In [166]:
X_train, X_valid, y_train, y_valid = train_test_split(df, label, random_state = 2, test_size = 0.3)

In [167]:
X_train.isnull().sum().sort_values(ascending=False)

B_rank                          2774
R_rank                          2462
finish_details                  1950
b_sub_odds                       741
b_ko_odds                        741
                                ... 
B_Stance                           0
B_wins                             0
B_win_by_TKO_Doctor_Stoppage       0
B_win_by_Submission                0
R_losses                           0
Length: 93, dtype: int64

## Building a baseline model

In [171]:
imp = SimpleImputer(strategy='most_frequent')
imp.fit(X_train[numeric_features])
imp.transform(X_train[numeric_features]).shape
X_valid[numeric_features] = imp.transform(X_valid[numeric_features])

cat_imp = SimpleImputer(strategy = 'most_frequent')
cat_imp.fit(X_train[categorical_features])
X_train[categorical_features] = cat_imp.transform(X_train[categorical_features])
X_valid[categorical_features] = cat_imp.transform(X_valid[categorical_features])
X_train.sample(10)


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,title_bout,weight_class,gender,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_SIG_STR_landed,B_avg_SIG_STR_pct,B_avg_SUB_ATT,B_avg_TD_landed,B_avg_TD_pct,B_longest_win_streak,B_losses,B_total_rounds_fought,B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins,B_Stance,B_Height_cms,B_Reach_cms,B_Weight_lbs,R_current_lose_streak,R_current_win_streak,R_draw,R_avg_SIG_STR_landed,R_avg_SIG_STR_pct,R_avg_SUB_ATT,R_avg_TD_landed,R_avg_TD_pct,R_longest_win_streak,R_losses,R_total_rounds_fought,R_total_title_bouts,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,R_age,B_age,lose_streak_dif,win_streak_dif,longest_win_streak_dif,win_dif,loss_dif,total_round_dif,total_title_bout_dif,ko_dif,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,constant_1,better_rank,finish,finish_details,finish_round,finish_round_time,total_fight_time_secs,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds,R_rank,B_rank
2933,Andrew Craig,Lyman Good,230.0,-270,230.000000,37.037037,7/15/2015,"San Diego, California, USA",USA,Blue,False,Welterweight,MALE,3,0,0,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,Orthodox,182.88,187.96,170,2,0,0,30.833333,0.446667,0.000000,0.500000,0.193333,2,3,16,0,0,1,1,1,0,0,3,Orthodox,185.42,193.04,170,29,30,2,0,-2,-3,3,-16,0,-1,0,-2.54,-5.08,-1,-30.833333,0.000000,-0.500000,0,1,neither,KO/TKO,Punch,2.0,3:37,517.0,350.0,162.0,1710.0,420.0,1050.0,400.0,NaN,NaN
2428,Paige VanZant,Bec Rawlings,-210.0,175,47.619048,175.000000,8/27/2016,"Vancouver, British Columbia, Canada",Canada,Red,False,Women's Strawweight,FEMALE,3,0,2,0,55.666667,0.506667,0.333333,0.333333,0.110000,2,1,7,0,0,0,1,0,1,0,2,Orthodox,167.64,162.56,125,1,0,0,52.750000,0.510000,1.000000,2.000000,0.330000,3,1,14,0,0,0,1,1,1,0,3,Orthodox,162.56,165.10,115,22,27,1,2,-1,-1,0,-7,0,-1,0,5.08,-2.54,-5,2.916667,-0.666667,-1.666667,0,1,Red,KO/TKO,Kick,2.0,0:17,317.0,NaN,325.0,NaN,1100.0,NaN,825.0,10.0,NaN
420,Alan Jouban,Jared Gooden,-150.0,120,66.666667,120.000000,11/21/2020,"Las Vegas, Nevada, USA",USA,Red,False,Welterweight,MALE,3,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,Orthodox,182.88,195.58,170,1,0,0,4.850000,0.500000,0.000000,0.590000,0.500000,3,5,24,0,0,0,3,4,0,0,7,Southpaw,182.88,185.42,170,38,26,-1,0,-3,-7,-5,-24,0,-4,0,0.00,10.16,-12,-4.850000,0.000000,-0.590000,1,1,neither,U-DEC,Punch,NaN,5:00,NaN,275.0,425.0,1000.0,1600.0,175.0,275.0,NaN,NaN
1240,Danny Henry,Dan Ige,130.0,-140,130.000000,71.428571,3/16/2019,"London, England, United Kingdom",United Kingdom,Blue,False,Featherweight,MALE,3,0,2,0,38.666667,0.580000,0.000000,1.666667,0.496667,2,1,7,0,0,0,1,1,0,0,2,Orthodox,170.18,180.34,145,0,2,0,47.500000,0.465000,1.500000,0.500000,0.165000,2,0,4,0,0,0,1,0,1,0,2,Orthodox,182.88,185.42,145,30,27,0,0,0,0,-1,3,0,1,-1,-12.70,-5.08,3,-8.833333,-1.500000,1.166667,0,1,neither,SUB,Rear Naked Choke,1.0,1:17,77.0,245.0,150.0,700.0,350.0,800.0,590.0,NaN,NaN
1868,Fabricio Werdum,Marcin Tybura,-300.0,250,33.333333,250.000000,11/18/2017,"Sydney, New South Wales, Australia",Australia,Red,False,Heavyweight,MALE,5,0,3,0,41.000000,0.502500,0.000000,1.000000,0.372500,3,1,11,0,0,0,1,2,0,0,3,Orthodox,190.50,198.12,249,0,1,0,45.000000,0.531429,0.285714,0.571429,0.213571,6,4,31,3,0,0,3,4,3,0,10,Orthodox,193.04,195.58,231,40,32,0,2,-3,-7,3,-20,-3,-2,-3,-2.54,2.54,8,-4.000000,-0.285714,0.428571,0,1,Red,U-DEC,Punch,5.0,5:00,1500.0,220.0,700.0,135.0,2500.0,475.0,500.0,2.0,8.0
1390,Tim Means,Ricky Rainey,-340.0,280,29.411765,280.000000,11/30/2018,"Las Vegas, Nevada, USA",USA,Red,False,Welterweight,MALE,3,1,0,0,19.000000,0.430000,0.000000,0.000000,0.000000,0,1,2,0,0,0,

In [173]:
base_model = DummyClassifier(random_state=2)
base_model.fit(X_train,y_train)

DummyClassifier(random_state=2)

In [174]:
preds = base_model.predict(X_valid)
accuracy_score(y_valid, preds)

0.5867937372362151